In [1]:
!pwd

/f/python_venv/End-to-End-Cancer-Classification-using-MLFlow-DVC/research


In [2]:
import os
os.chdir("../")
!pwd

/f/python_venv/End-to-End-Cancer-Classification-using-MLFlow-DVC


In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/karan-1700/End-to-End-Cancer-Classification-using-MLFlow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "karan-1700"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "NaN"

In [4]:
import tensorflow as tf


In [5]:
# load the trained model
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
# entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_to_model: Path
    training_data_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
# configuration manager

from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> ModelEvaluationConfig:
        evaluation_config = ModelEvaluationConfig(
                path_to_model="artifacts/training/model.h5",
                training_data_path="artifacts/data_ingestion/Chest-CT-Scan-data",
                all_params=self.params,
                mlflow_uri="https://dagshub.com/karan-1700/End-to-End-Cancer-Classification-using-MLFlow-DVC.mlflow",
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE,
        )

        return evaluation_config


In [ ]:
# component

from pathlib import Path
from urllib.parse import urlparse
import tensorflow as tf
import mlflow
import mlflow.keras


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluate(self):
        self.model = self.load_model(path=self.config.path_to_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        # self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json", data=scores))


    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")



In [17]:
# pipeline

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate()
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-04 21:48:46,085] [INFO] [common.read_yaml:33] [yaml file: config\config.yaml loaded successfully]
[2025-08-04 21:48:46,085] [INFO] [common.read_yaml:33] [yaml file: params.yaml loaded successfully]
[2025-08-04 21:48:46,089] [INFO] [common.create_directories:52] [Created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 13s 2s/step - loss: 29.7200 - accuracy: 0.4314


2025/08/04 21:49:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-04 21:49:04,931] [WARNING] [save.__init__:274] [Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\padhi\AppData\Local\Temp\tmpowvsayz2\model\data\model\assets
[2025-08-04 21:49:07,207] [INFO] [builder_impl.copy_assets_to_destination_dir:797] [Assets written to: C:\Users\padhi\AppData\Local\Temp\tmpowvsayz2\model\data\model\assets]


f:\python_venv\mlflow\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2025/08/04 21:49:58 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [ ]:
import dagshub
dagshub.init(repo_owner='karan-1700', repo_name='End-to-End-Cancer-Classification-using-MLFlow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

